In [ ]:
# ==========================================
# VAE anomaly detection (PyTorch) with manual threshold option
# Dataset: ai4i2020.csv
# - Train: ONLY normal
# - VAL (balanced): dùng để chọn ngưỡng (manual / F1 / Precision≥target)
# - TEST Balanced & Imbalanced dùng cùng ngưỡng
# ==========================================

import os, random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    precision_recall_fscore_support, precision_recall_curve
)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# ------------------------------
# 0) Reproducibility & determinism
# ------------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ------------------------------
# 1) Load & preprocess
# ------------------------------
csv_path = "../../data/ai4i2020.csv"   # đổi thành "/mnt/data/ai4i2020.csv" nếu cần
df = pd.read_csv(csv_path)

print("Tổng số dòng:", len(df))
print("Phân phối nhãn Machine failure:\n", df["Machine failure"].value_counts())
print("Tỷ lệ (%):\n", df["Machine failure"].value_counts(normalize=True) * 100)

drop_cols = ['UDI', 'Product ID', 'Type', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']
X_df = df.drop(columns=drop_cols)
y = df['Machine failure'].to_numpy().astype(int)

scaler = StandardScaler()
X_all = scaler.fit_transform(X_df).astype(np.float32)
input_dim = X_all.shape[1]

# ------------------------------
# 2) Build splits (auto theo thực tế n_anom≈339)
# ------------------------------
normal_idx = np.where(y == 0)[0]
anom_idx   = np.where(y == 1)[0]
n_normal, n_anom = len(normal_idx), len(anom_idx)
print(f"\nNormal: {n_normal} | Anomaly: {n_anom}")

# xáo trộn để không phụ thuộc thứ tự
rng = np.random.default_rng(SEED)
rng.shuffle(normal_idx); rng.shuffle(anom_idx)

# Train normal
TRAIN_NORMAL = min(8000, n_normal - 2000) if n_normal > 2000 else max(1000, n_normal // 2)
train_norm_idx = normal_idx[:TRAIN_NORMAL]

# Validation balanced
VAL_ANOM = min(150, n_anom // 2)
VAL_NORM = VAL_ANOM
val_anom_idx = anom_idx[:VAL_ANOM]
val_norm_idx = normal_idx[TRAIN_NORMAL : TRAIN_NORMAL + VAL_NORM]

# Test balanced
TEST_ANOM = min(200, n_anom - VAL_ANOM)
TEST_NORM_BAL = TEST_ANOM
test_bal_anom_idx = anom_idx[VAL_ANOM : VAL_ANOM + TEST_ANOM]
test_bal_norm_idx = normal_idx[TRAIN_NORMAL + VAL_NORM :
                               TRAIN_NORMAL + VAL_NORM + TEST_NORM_BAL]

# Test imbalanced ~4:1
TEST_IMB_NORM = min(800, n_normal - (TRAIN_NORMAL + VAL_NORM + TEST_NORM_BAL))
test_imb_norm_idx = normal_idx[TRAIN_NORMAL + VAL_NORM + TEST_NORM_BAL :
                               TRAIN_NORMAL + VAL_NORM + TEST_NORM_BAL + TEST_IMB_NORM]
test_imb_anom_idx = test_bal_anom_idx  # dùng cùng anomaly để so công bằng

def take(idx):
    return X_all[idx], y[idx]

X_train, y_train = take(train_norm_idx)           # toàn 0 (normal only)
X_val   = np.vstack([X_all[val_norm_idx], X_all[val_anom_idx]])
y_val   = np.hstack([np.zeros(len(val_norm_idx), dtype=int),
                     np.ones (len(val_anom_idx), dtype=int)])

X_test_bal = np.vstack([X_all[test_bal_norm_idx], X_all[test_bal_anom_idx]])
y_test_bal = np.hstack([np.zeros(len(test_bal_norm_idx), dtype=int),
                        np.ones (len(test_bal_anom_idx), dtype=int)])

X_test_imb = np.vstack([X_all[test_imb_norm_idx], X_all[test_imb_anom_idx]])
y_test_imb = np.hstack([np.zeros(len(test_imb_norm_idx), dtype=int),
                        np.ones (len(test_imb_anom_idx), dtype=int)])

print("\n--- Split summary (auto) ---")
print(f"Train normal size          : {X_train.shape[0]}")
print(f"VAL balanced (norm/anom)   : {len(val_norm_idx)} / {len(val_anom_idx)}")
print(f"TEST balanced (norm/anom)  : {len(test_bal_norm_idx)} / {len(test_bal_anom_idx)}")
print(f"TEST imbalanced (norm/anom): {len(test_imb_norm_idx)} / {len(test_imb_anom_idx)}")

# ------------------------------
# 3) Define VAE
# ------------------------------
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=8, hidden=128):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
        )
        self.enc_mu     = nn.Linear(hidden, latent_dim)
        self.enc_logvar = nn.Linear(hidden, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, input_dim)  # tái tạo z-score
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar); eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, lv = self.enc_mu(h), self.enc_logvar(h)
        z = self.reparameterize(mu, lv)
        xhat = self.decoder(z)
        return xhat, mu, lv, z

def vae_loss_fn(x, xhat, mu, logvar, recon="mse", beta=1.0):
    if recon == "mse":
        recon_loss = nn.functional.mse_loss(xhat, x, reduction="mean")
    else:
        recon_loss = nn.functional.l1_loss(xhat, x, reduction="mean")
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + beta * kl, recon_loss, kl

# ------------------------------
# 4) Train VAE (ONLY normal)
# ------------------------------
vae = VAE(input_dim=input_dim, latent_dim=8, hidden=128).to(device)
opt = optim.Adam(vae.parameters(), lr=1e-3)

batch_size = 128
epochs     = 40   # có thể tăng nếu có GPU
train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train)),
                          batch_size=batch_size, shuffle=True, drop_last=False)

for ep in range(1, epochs+1):
    vae.train()
    total = recon_total = kl_total = 0.0
    steps = 0
    for (xb,) in train_loader:
        xb = xb.to(device)
        opt.zero_grad()
        xhat, mu, lv, z = vae(xb)
        loss, rl, kl = vae_loss_fn(xb, xhat, mu, lv, recon="mse", beta=1.0)
        loss.backward(); opt.step()
        total += loss.item(); recon_total += rl.item(); kl_total += kl.item(); steps += 1
    if ep == 1 or ep % 5 == 0:
        print(f"[VAE] Epoch {ep:3d}/{epochs} | loss={total/steps:.6f} | recon={recon_total/steps:.6f} | kl={kl_total/steps:.6f}")

# ------------------------------
# 5) Scoring: reconstruction error (MSE)
# ------------------------------
@torch.no_grad()
def recon_error(x_np: np.ndarray, model: VAE, agg="mse"):
    model.eval()
    xt = torch.from_numpy(x_np).to(device)
    xhat, _, _, _ = model(xt)
    if agg == "mse":
        err = torch.mean((xhat - xt)**2, dim=1)
    else:
        err = torch.mean(torch.abs(xhat - xt), dim=1)
    return err.detach().cpu().numpy()

val_scores  = recon_error(X_val, vae, agg="mse")

# ------------------------------
# 6) Threshold selection (manual / f1 / p_at)
# ------------------------------
MODE = "manual"    # "manual" | "f1" | "p_at"
THR_MANUAL = 0.60  # <-- tự đặt ngưỡng theo kinh nghiệm/target (thử 0.4~0.8)
TARGET_P   = 0.60  # dùng khi MODE="p_at"

def best_f1_threshold(y_true, scores, percentiles=np.linspace(50, 99.5, 200)):
    thrs = np.percentile(scores, percentiles)
    best_f1, best_thr = -1.0, None
    for t in thrs:
        y_pred = (scores >= t).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, labels=[0,1], average=None, zero_division=0
        )
        if f1[1] > best_f1:
            best_f1, best_thr = float(f1[1]), float(t)
    return best_thr, best_f1

def threshold_for_precision(y_true, scores, target_p=0.60):
    p, r, thr = precision_recall_curve(y_true, scores)
    idx = np.where(p[:-1] >= target_p)[0]
    if len(idx) == 0:
        # fallback: 95th percentile nếu không đạt precision mục tiêu
        return float(np.percentile(scores, 95)), float(p[1] if len(p)>1 else 0.0), float(r[1] if len(r)>1 else 0.0)
    i = idx[0]
    return float(thr[i]), float(p[i]), float(r[i])

if MODE == "manual":
    thr = float(THR_MANUAL)
    pct = (val_scores <= thr).mean() * 100.0
    print(f"\n[VAL] Manual threshold selected: thr={thr:.6f} (~percentile {pct:.2f}%)")
elif MODE == "f1":
    thr, best_f1 = best_f1_threshold(y_val, val_scores)
    print(f"\n[VAL balanced] Best F1(Class 1)={best_f1:.3f} at threshold={thr:.6f}")
else:  # MODE == "p_at"
    thr, p_at, r_at = threshold_for_precision(y_val, val_scores, TARGET_P)
    print(f"\n[VAL balanced] Threshold for Precision≥{TARGET_P:.2f}: thr={thr:.6f} (P={p_at:.3f}, R={r_at:.3f})")

# ------------------------------
# 7) Evaluate on TEST sets
# ------------------------------
def evaluate(name: str, X_np: np.ndarray, y_true: np.ndarray, thr: float):
    scores = recon_error(X_np, vae, agg="mse")
    y_pred = (scores >= thr).astype(int)
    print(f"\n===== {name} @thr={thr:.6f} =====")
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\nClassification Report:\n", classification_report(y_true, y_pred, digits=4))
    print("ROC AUC (scores):", roc_auc_score(y_true, scores))

evaluate("TEST Balanced",   X_test_bal, y_test_bal, thr)
evaluate("TEST Imbalanced", X_test_imb, y_test_imb, thr)

# ------------------------------
# 8) (Optional) Quick sweep ngưỡng để so sánh
# ------------------------------
candidates = [np.percentile(val_scores, p) for p in [50, 60, 70, 80, 85, 90, 95, 97.5, 99]]
print("\n>>> Quick sweep on percentile-based thresholds (from VAL):")
for t in candidates:
    print(f"\n-- Try thr={t:.6f} on TEST Balanced --")
    evaluate("TEST Balanced", X_test_bal, y_test_bal, t)
    print(f"\n-- Try thr={t:.6f} on TEST Imbalanced --")
    evaluate("TEST Imbalanced", X_test_imb, y_test_imb, t)


Device: cpu
Tổng số dòng: 10000
Phân phối nhãn Machine failure:
 Machine failure
0    9661
1     339
Name: count, dtype: int64
Tỷ lệ (%):
 Machine failure
0    96.61
1     3.39
Name: proportion, dtype: float64

Normal: 9661 | Anomaly: 339

--- Split summary (auto) ---
Train normal size          : 7661
VAL balanced (norm/anom)   : 150 / 150
TEST balanced (norm/anom)  : 189 / 189
TEST imbalanced (norm/anom): 800 / 189
[VAE] Epoch   1/40 | loss=0.813664 | recon=0.702319 | kl=0.111345
[VAE] Epoch   5/40 | loss=0.515744 | recon=0.229941 | kl=0.285803
[VAE] Epoch  10/40 | loss=0.513013 | recon=0.226496 | kl=0.286517
[VAE] Epoch  15/40 | loss=0.511682 | recon=0.223840 | kl=0.287842
[VAE] Epoch  20/40 | loss=0.510439 | recon=0.223741 | kl=0.286698
[VAE] Epoch  25/40 | loss=0.512371 | recon=0.224506 | kl=0.287865
[VAE] Epoch  30/40 | loss=0.508690 | recon=0.223101 | kl=0.285590
[VAE] Epoch  35/40 | loss=0.507612 | recon=0.223040 | kl=0.284572
[VAE] Epoch  40/40 | loss=0.507629 | recon=0.224387 